In [ ]:
!pip install PyMySQL
!pip install pymongo

In [2]:
import sys
import pymysql
from pymongo import MongoClient
import pymongo
import datetime

import getNYT as gn
import getCNN as gc
import getBBC as gb

<h3>Insert article list to MySQL</h3>

In [ ]:
try:
    conn = pymysql.connect(host='10.120.28.52',
                       user='gp3',
                       password='Group_03',
                       db='gp3',
                       charset='utf8',
                       autocommit=True,
                       cursorclass=pymysql.cursors.DictCursor)
    c= conn.cursor()
except:
    print('Cannot connect to MySQL server!')

In [ ]:
# CNN test
cats = gc.getCatLinks()

for cat in cats:
    okcount = 0
    ngcount = 0
#     print(cat)
    
    for article in gc.getArticlesList(cat):
        try:
            c.execute("""INSERT INTO cklist(title,source,url) VALUES (%(title)s,%(source)s,%(url)s)""",article)
            print('Insert successful: %s'%(article['url']))
            okcount += 1
        except:
            print('%s Error: %s'%(sys.exc_info()[0],article['url']))
            ngcount += 1
print('OK:%s ; NG:%s'%(okcount,ngcount))

In [ ]:
# NYT test
articles = gn.getArticlesList(365)
okcount = 0
ngcount = 0

for article in articles:
#     print(cat)
    try:
        c.execute("""INSERT INTO cklist(title,source,url) VALUES (%(title)s,%(source)s,%(url)s)""",article)
        print('Insert successful: %s'%(article['url']))
        okcount += 1
    except:
        print('%s Error: %s'%(sys.exc_info()[0],article['url']))
        ngcount += 1
print('OK:%s ; NG:%s'%(okcount,ngcount))

In [ ]:
# BBC test
cats = gb.getCatLinks()
okcount = 0
ngcount = 0

for cat in cats:
#     print(cat)
    for article in gb.getArticlesList(cat):
        try:
            c.execute("""INSERT INTO cklist(title,source,url) VALUES (%(title)s,%(source)s,%(url)s)""",article)
            print('Insert successful: %s'%(article['url']))
            okcount += 1
        except:
            print('%s Error: %s'%(sys.exc_info()[0],article['url']))
            ngcount += 1
print('OK:%s ; NG:%s'%(okcount,ngcount))

<h3>Get url list from MySQL and insert to MongoDB after crawling</h3>

In [4]:
# Get connection to MySQL@10.120.28.52
try:
    conn = pymysql.connect(host='10.120.28.52',
                           user='gp3',
                           password='Group_03',
                           db='gp3',
                           charset='utf8',
                           autocommit=True,
                           cursorclass=pymysql.cursors.DictCursor)
    c = conn.cursor()
except:
    print('Cannot connect to MySQL server!')

In [11]:
# [DEBUG] Change back tag from R(already-read) to N(not-be-crawled)
result = c.execute("""SELECT url FROM cklist WHERE source = 'NYT' AND tag = 'R'""")
print('Get %s records.'%(result))

urls = []
for url in c:
    urls.append(url['url'])
print(urls)

for url in urls:
    c.execute("""UPDATE cklist SET tag = 'N' WHERE url = %s""",url)

Get 0 records.
[]


In [ ]:
# Get connection to MongoDB@10.120.28.55
try:
    client = MongoClient('10.120.28.55', 27017)
    db = client['news']
    collection = db['news']
except:
    print('Cannot connect to MySQL server!')

In [ ]:
# Get unread url from cklist table
records = c.execute("""SELECT url FROM cklist WHERE source = 'NYT' AND tag = 'N' LIMIT 5""")
print('Get %s records.'%(records))

urls = []
for record in c:
    urls.append(record['url'])
print(urls)

In [ ]:
# Get article content and insert into MongoDB
for url in urls:
    
    art_dict = gn.getArticle(url)
    
    print('%s has %s paras'%(art_dict['title'],len(art_dict['content'])))
    
    if (len(art_dict['content']) > 1):
        collection.insert_one(art_dict)
        print('[Insert %s paras ok!] %s'%(len(art_dict['content']),art_dict['title']))
    else:
        print('[%s] is empty' % (art_dict['url']))
        
    c.execute("""UPDATE cklist SET tag = 'R' WHERE url = %s""", url)